<h1>Tutorial</h1><br><h4>Beat tracking example</h4>

In [4]:
import librosa

# 1. 음성 파일 불러오기
filename = librosa.example('nutcracker')

# 2. 오디오를 '파형' format으로 불러옴
## time domain의 오디오 데이터 y, Numpy floating array로 저장
## sampling rate = sr
y, sr = librosa.load(filename)

# 3. beat tracker 실행
## tempo : bpm이라 생각하면 편함
## beat_frames : audio를 window로 쪼갠다. window의 길이는 hop_length로, default가 512개의 샘플 개수인듯 하다.
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)

print('Estimated tempo: {:.2f} beats per minute'.format(tempo))

# 4. beat_frames를 beat_times로 변환
## beat_times : beat events가 발견된 time stamp
beat_times = librosa.frames_to_time(beat_frames, sr=sr)

print(beat_frames, beat_times)

Estimated tempo: 107.67 beats per minute
[  51   74  100  124  149  173  198  221  247  271  295  319  344  368
  393  416  440  462  486  508  531  554  578  601  624  647  670  693
  716  739  762  786  810  832  856  878  901  924  947  970  993 1016
 1039 1062 1085 1108 1131 1155 1178 1201 1225 1249 1273 1297 1321 1344
 1369 1393 1415 1437 1460 1483 1505 1527 1550 1573 1595 1618 1642 1665
 1689 1712 1736 1759 1783 1806 1829 1853 1876 1900 1924 1947 1971 1994
 2018 2042 2065 2088 2110 2132 2155 2177 2200 2223 2244 2266 2290 2313
 2336 2359 2381 2404 2427 2451 2474 2498 2522 2545 2568 2592 2615 2639
 2662 2684 2706 2729 2752 2775 2797 2819 2842 2864 2887 2910 2933 2956
 2979 3003 3027 3051 3075 3100 3125 3150 3174 3199 3223 3246 3269 3295
 3318 3343 3367 3391 3415 3439 3462 3485 3509 3534 3560 3586 3612 3637
 3664 3689 3715 3740 3766 3791 3817 3842 3866 3891 3916 3940 3965 3990
 4014 4038 4063 4087 4112 4136 4160 4184 4208 4232 4256 4280 4304 4327
 4352 4375 4400 4423 4447 4471 4495 

<h4>Advance usage</h4>

In [6]:
import numpy as np
#import librosa

y, sr = librosa.load(librosa.ex('nutcracker'))

# 보통 음원의 sr = 22052 Hz, sample 512의 경우 대략 23ms.
## hop_length가 음원 분석에서의 해상도를 결정하는 파라미터.
hop_length =512

# 현악기의 소리와 타악기 소리 분리
y_harmonic, y_percussive = librosa.effects.hpss(y)

# 분리한 타악기의 beat 분석
tempo, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)

# MFCC 계산
mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)

# MFCC의 1차 미분(차분), 성분을 delta feature라고 부른다.
mfcc_delta = librosa.feature.delta(mfcc)

#두 beat events를 결합 및 싱크
## vstack = vertical stack, 즉, 수직으로 쌓는 것.
## default는 mean value지만 median도 선택 가능
beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]),
                                    beat_frames)

# 크로마그램은 음악 신호를 짧은 길이(수 십 ~ 수 백 ms)의 프레임 단위로 나누고 
# 각 프레임에서 스펙트럼을 구하고 옥타브 단위의 차이나는 음악 스펙트럼 성분들을 가산하여 음악의 
# 전체 스펙트럼 성분들을 하나의 옥타브 안으로 접어서 표현한 것
chromagram = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr)

beat_chroma = librosa.util.sync(chromagram,
                                beat_frames,
                                aggregate=np.median)

beat_features = np.vstack([beat_chroma, beat_mfcc_delta])